setup code goes here

In [1]:
import os
import shutil
import asyncio
import tempfile
import threading
import contextlib

import unittest.mock as mock

import synapse.glob as s_glob
import synapse.common as s_common
import synapse.cortex as s_cortex
# import synapse.lib.cli as s_cli
import synapse.lib.cmdr as s_cmdr

import synapse.tests.utils as s_t_utils
print(f'Current pid: {os.getpid()}')
current_thread = threading.current_thread()
# print(f'Current thread / thread.ident: {current_thread}')
# print(type(current_thread))
loop = s_glob.initloop()
# print(f'Current loop: {loop} / {id(loop)}')
glob_thrd = s_glob._glob_thrd
# print(f'loop thread: {glob_thrd}')

@contextlib.asynccontextmanager
async def getTempCoreProxy():
    with s_common.getTempDir() as dirn:
        # print(f'Got tempdir: {dirn}')
        # print('getting cortex....')
        async with await s_cortex.Cortex.anit(dirn) as core:
            # print(f'Got cortex: {core}')
            # print('Getting local proxy')
            # core.conf['storm:log'] = True
            async with core.getLocalProxy() as prox:
                # print(f'Got proxy: {prox}')
                yield prox
                # print('closing down proxy/local dmon')
            # print('closing down cortex')
        # print('tearing down tempdir')
    # print('done with getTempCortex')

acm = getTempCoreProxy()
proxy = await acm.__aenter__()
# print(proxy, proxy.isfini)

# print('preloading some data...')
async for mesg in await proxy.storm('[inet:ipv4=1.2.3.4 inet:ipv4=8.8.8.8]'):
    pass
    #print(mesg)
    
cmdr = await s_cmdr.getItemCmdr(proxy)
cmdr.echoline = True
# print(f'Got cmdr: {cmdr}')

Current pid: 26912


Execute a storm command!

In [2]:
await cmdr.runCmdLine('storm inet:ipv4')
# cmdr.runCmdLoop()
# await cmdr.asyncRunCmdLoop()

cli> storm inet:ipv4

inet:ipv4=1.2.3.4
        .created = 2018/12/07 22:38:02.337
        :asn = 0
        :loc = ??
        :type = unicast
inet:ipv4=8.8.8.8
        .created = 2018/12/07 22:38:02.341
        :asn = 0
        :loc = ??
        :type = unicast
complete. 2 nodes in 5 ms (400/sec).


See how cool that was!

In [3]:
await acm.__aexit__(None, None, None)
# print(proxy, proxy.isfini)#
    

connection closed...
